In [1]:
import torch
import torch.nn as nn
import math

In [2]:
num_samples = 1000
numbers = torch.randint(1, 1001, (num_samples, 1)).float()
squares = torch.sqrt(numbers)

In [3]:
numbers[0], squares[0] # sample data

(tensor([332.]), tensor([18.2209]))

In [6]:
class SqrtModel(nn.Module):
    def __init__(self):
        super(SqrtModel, self).__init__()
        self.hidden = nn.Linear(1, 10)
        self.relu = nn.ReLU()
        self.out = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.log1p(x)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.out(x)
        return torch.exp(x)

In [7]:
model = SqrtModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(numbers)
    loss = criterion(outputs, squares)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

Epoch 100/1000, Loss: 0.6860
Epoch 200/1000, Loss: 0.4673
Epoch 300/1000, Loss: 0.2893
Epoch 400/1000, Loss: 0.1616
Epoch 500/1000, Loss: 0.0822
Epoch 600/1000, Loss: 0.0386
Epoch 700/1000, Loss: 0.0171
Epoch 800/1000, Loss: 0.0077
Epoch 900/1000, Loss: 0.0040
Epoch 1000/1000, Loss: 0.0027


In [8]:
def absolute_percentage_error(actual, predicted):
    if actual == 0:
        return 0
    ape = abs((actual - predicted) / actual) * 100
    return ape

In [10]:
def find_square_root(model, number):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor([[number]], dtype=torch.float32)
        predicted_sqrt = model(input_tensor)
        actual_sqrt = torch.sqrt(input_tensor)
    return actual_sqrt.item(), predicted_sqrt.item()

number = 845714
actual_sqrt, predicted_sqrt = find_square_root(model, float(number))
print(f"Number: {number}, Predicted Square Root: {predicted_sqrt:.4f}, True Square Root: {actual_sqrt:.4f}")
error  = absolute_percentage_error(actual_sqrt, predicted_sqrt)
print(f"Absolute Percentage Error: {error:.2f}%")

Number: 845714, Predicted Square Root: 918.8904, True Square Root: 919.6271
Absolute Percentage Error: 0.08%
